In [7]:
# Script de Preparación de Datos
###################################

import pandas as pd
import numpy as np
import lifetimes
import os

# Leemos los archivos csv

def read_file_csv(filename):
    df = pd.read_csv(os.path.join('../data/raw/', filename))
    print(filename, ' cargado correctamente')
    return df

# Funciones para manejon de outliers

def find_boundaries(df, variable, q1=0.05, q2=0.95):

    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1)
    upper_boundary = df[variable].quantile(q2)
    return upper_boundary, lower_boundary

def capping_outliers(df, variable):

    # aplica los límites de los quantiles
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                            np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

# Generación del archivo rfm

def data_preparation(df):

    # Nos quedamos los valores de Precio y Cantidad mayores a cero
    df = df[df['Quantity'] > 0 ]
    df = df[df['UnitPrice'] > 0]
    df = df[~df['InvoiceNo'].str.contains("C",na=False)] # drop returned items

    # Eliminación de outliers
    df.dropna(inplace=True)

    # Aplicando las reglas para manejo de outliers
    capping_outliers(df,'UnitPrice')
    capping_outliers(df,'Quantity')

    # Filtra valores 'Usamos solo del Reino Unido'
    df = df[df.Country == 'United Kingdom']

    # Se crea la columna 'Total Price'
    df['Total Price'] = df['UnitPrice'] * df['Quantity']

    # Creating Summary Dataset
    clv = lifetimes.utils.summary_data_from_transaction_data(df,'CustomerID',
                                                             'InvoiceDate',
                                                             'Total Price',
                                                             observation_period_end='2011-12-09')

    # we want only customers shopped more than 2 times
    clv = clv[clv['frequency']>1]

    return clv

# Exportamos la tabla rfm

def data_exporting(df, filename):
    dfp.to_csv(os.path.join('../data/processed/', filename))
    print(filename, 'exportado correctamente en la carpeta processed')
    

PRUEBAS

In [5]:
df = read_file_csv('Online_Retail.csv')

Online_Retail.csv  cargado correctamente


In [8]:
data_preparation(df)

,frequency,recency,T,monetary_value
CustomerID,,,,
12747.0,10.0,367.0,369.0,375.725000
12748.0,112.0,373.0,373.0,257.314911
12749.0,3.0,210.0,213.0,999.106667
12820.0,3.0,323.0,326.0,256.573333
12823.0,4.0,222.0,296.0,252.450000
...,...,...,...,...
18263.0,2.0,227.0,252.0,467.580000
18272.0,5.0,244.0,246.0,485.688000
18273.0,2.0,255.0,257.0,71.400000
